# Idea
### CLIP1:
In this version I am trying to use only image and designation, I'm applying
Explicit Cross-modal Interaction from
[Hate-CLIPper: Multimodal Hateful Meme Classification based on Cross-modal Interaction of CLIP Features](https://arxiv.org/pdf/2210.05916.pdf)

### CLIP2:
diff: added a dropout layer to reduce overfitting


### CLIP3:
diff: if description exists, we get the mean of the features of description and designation

# Hyperparameters

In [1]:
BATCH_SIZE=128#  8 for 13.8 gb usage, 6 for less than 12 gb usage
NUM_CLASSES=27

CLIP_FEATURE_SIZE=768

#testing
# train_percentage=0.0001
# valid_percentage=0.0001

train_percentage=0.9
valid_percentage=0.1
#(test_percentage takes the rest)

google_colab=True
force_cpu=False
train=False

# Imports


In [2]:
%pip install transformers pandas tqdm scikit-learn imageio matplotlib wget plotly dash

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 29.5 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=48bce7876436f401482fbe303dc8317c53f6491650addcbc919cb44dae106e09
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [3]:
from transformers import CLIPProcessor, CLIPModel,CLIPFeatureExtractor
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
import torchvision

import imageio

from tqdm.notebook import tqdm
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import f1_score

import zipfile
import os
import copy

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

from datetime import datetime


In [4]:
if google_colab:
  # mount the drive where your dataset is availabledevice = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  from google.colab import drive
  drive.mount('/content/drive')
  filepath='/content/drive/MyDrive/datasets/multimodal_product_classification/' # add your own path. Where to save the dataset

  if not os.path.exists('datasets'):
    os.makedirs('datasets')
    with zipfile.ZipFile(filepath+'images.zip', 'r') as zip_ref:
        zip_ref.extractall('datasets')

  datasets_path="/content/datasets/"
  save_directory="/content/drive/MyDrive/Lessons/Models/multimodal_classification/"

else:
  import wget
  if not os.path.exists('datasets'):
    os.makedirs('datasets')
    output_directory="datasets"
    csv_zip = wget.download("https://nextcloud.its-tps.fr/s/BTpB4SC93NreZxg/download/csv_data.zip",out=output_directory)
    images_zip=wget.download("https://nextcloud.its-tps.fr/s/fgBxQczEAZ7ws8J/download/images.zip",out=output_directory)

    with zipfile.ZipFile(output_directory+'/csv_data.zip', 'r') as zip_ref:
      zip_ref.extractall('datasets')
    with zipfile.ZipFile(output_directory+'/images.zip', 'r') as zip_ref:
        zip_ref.extractall('datasets')
  filepath=os.getcwd()+'/datasets/'
  save_directory='../models/'
  datasets_path=filepath


Mounted at /content/drive


In [5]:
if force_cpu:
    device = torch.device("cpu")
else:
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# Preprocessing

In [6]:
# Apply cleaning function to the 'designation' column
def clip_to_max_tokenize(text):
    text.split(" ",maxsplit=70)
    text=text[:70]
    text="".join(text)
    return(text)

plot a scatter plot of the representation in term of word as x number of word in designation and as y number of word in description.
if we see that sometimes x is low and y is big, we could fill designation with description

In [7]:
y_train = pd.read_csv(filepath+'Y_train.csv').fillna("")
y_train=y_train.drop(columns="Unnamed: 0")
encoded_df=pd.get_dummies(y_train, columns=['prdtypecode'])
y_train_categorical = encoded_df.values.tolist()
y_train


,prdtypecode
0,10
1,2280
2,50
3,1280
4,2705
...,...
84911,40
84912,2583
84913,2280
84914,1560


In [8]:
y_train_out=np.argmax(y_train_categorical,1)
y_train_out

array([ 0, 18,  2, ..., 18, 13, 21])

In [9]:
label_dict={}
i=0
while len(label_dict)<27:
    key=y_train_out[i]
    if key not in label_dict.keys():
        value=y_train[ 'prdtypecode'][i]
        label_dict[key] = value
    i+=1

label_dict

{0: 10,
 18: 2280,
 2: 50,
 7: 1280,
 25: 2705,
 21: 2522,
 22: 2582,
 13: 1560,
 8: 1281,
 14: 1920,
 19: 2403,
 4: 1140,
 23: 2583,
 6: 1180,
 9: 1300,
 20: 2462,
 5: 1160,
 16: 2060,
 1: 40,
 3: 60,
 12: 1320,
 11: 1302,
 17: 2220,
 26: 2905,
 24: 2585,
 15: 1940,
 10: 1301}


# Data loading

In [10]:
def image_to_tensor(image):
    img=torch.from_numpy(image).float()
    out=img.permute(2,0,1)
    return(out)

In [11]:
class ImageTextDataLoader(Dataset):
    """Title, Description and Image dataset."""

    def __init__(self, dataframe, image_dir):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            image_dir (string): Directory with all the images.
        """
        self.df = dataframe
        self.image_dir = image_dir

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        selected_df=self.df.iloc[idx]

        image_name="image_"+str(selected_df["imageid"])+"_product_"+str(selected_df["productid"])+".jpg"
        filepath=os.path.join(self.image_dir,image_name)
        image_arr = imageio.v3.imread(filepath)

        designation=selected_df['designation']
        description=selected_df['description']
        label=torch.tensor(selected_df['labels'], dtype=torch.float,device=device)

        return [designation,description,image_arr,label]


In [12]:
Clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)
Clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [13]:
def get_images_features(images):
    inputs = Clip_processor(images=images, return_tensors="pt").to(device)
    image_features = Clip_model.get_image_features(**inputs).to(device)
    del inputs
    return(image_features)

def get_text_features(texts):
    inputs = Clip_processor(text=texts, padding=True, return_tensors="pt").to(device)
    text_features = Clip_model.get_text_features(**inputs).to(device)
    del inputs
    return(text_features)

# Model Definition

way 0 : concat the layer
way 1 (future): cross modal concatenation

In [14]:
print(Clip_model.device)

cuda:0


In [15]:
class ClassificationHead(nn.Module):
   def __init__(self, input_dim, num_classes):
       super(ClassificationHead, self).__init__()

       self.head=nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(input_dim,128),
            nn.ReLU(),
            nn.Linear(128,num_classes),
       )
       #self.softmax = nn.Softmax(dim=-1)

   def forward(self, x):
       x = self.head(x)
       return (x)


## eval on test(/eval) dataset

In [16]:
# Load data
X_eval = pd.read_csv(filepath+'X_test.csv').fillna("")
X_eval=X_eval.drop(columns="Unnamed: 0")
X_eval['designation'] = X_eval['designation'].fillna('').apply(clip_to_max_tokenize)
X_eval['description'] = X_eval['description'].fillna('').apply(clip_to_max_tokenize)

X_eval["labels"]=np.zeros((len(X_eval["designation"]),1))

In [17]:
eval_dataset=ImageTextDataLoader(X_eval,datasets_path+"/images/image_test")
print(len(eval_dataset))
eval_dataloader = DataLoader(eval_dataset, batch_size=BATCH_SIZE, shuffle=False)


13812


In [18]:
saved_state_dict = torch.load(filepath+"/clip3-2024-01-15-5.ckpt")
model=ClassificationHead(CLIP_FEATURE_SIZE**2,NUM_CLASSES).to(device)
model.load_state_dict(saved_state_dict)
model.to(device)
model.eval()
torch.no_grad()

In [19]:
@torch.no_grad
def eval_model(model,eval_dataloader):

    loop_on_batch=tqdm(eval_dataloader,leave=False,position=0,ncols=800)
    preds=[]
    for batch in loop_on_batch:
        # print(batch)
        designation,description,image_arr,_=batch

        # Iterate over data.
        description_features=[]

        for i in range(len(description)):
            #get the description that are not null
            if len(description[i])>=1:
                # print(f'{description[i]=}')
                description_feature=get_text_features(description[i])
                description_features.append(description_feature[0])
            else:
                description_features.append([])
        images_features=get_images_features(image_arr).unsqueeze(2)
        designation_features=get_text_features(designation).unsqueeze(2)


        if len(description_features[0])>0:
            designation_features[0]=(designation_features[0]+description_features[0].unsqueeze(1))/2

        #HERE implement fusion model of designation, description and image_arr
        input_features=torch.matmul(images_features[0] , designation_features[0].T).flatten().to(device).unsqueeze(0)

        for i in range(1,images_features.size(dim=0)):
            # Perform the multiplication and append the result to the results array
            if len(description_features[i])>0:
                designation_features[i]=(designation_features[i]+description_features[i].unsqueeze(1))/2

            feature_interaction_matrix=torch.matmul(images_features[i], designation_features[i].T)
            input_feature=(feature_interaction_matrix).flatten().to(device)
            input_features = torch.cat((input_features, input_feature.unsqueeze(0)), dim=0)

        # forward
        pred=model(input_features)
        # print(pred)
        preds.append(pred)


    return preds

In [20]:
preds=eval_model(model,eval_dataloader)

  0%|                                                                                                         …

KeyboardInterrupt: 

In [ ]:
preds2=[]
for pred in preds:
    preds2.extend(torch.max(pred,1)[1].tolist())


In [ ]:
def switch_labels(label):
    val=label_dict[label]
    return(val)

In [ ]:
df_preds=pd.Dataframe(preds2)
df_preds["0"]=df_preds["0"].apply(switch_labels)
df_preds["prdtypecode"]=df_preds["0"]
df_preds=df_preds.drop(columns="0")
df_preds

In [ ]:
df_preds=df_preds.drop(columns="Unnamed: 0")
df_preds

In [ ]:
df_preds=df_preds.set_index(df_preds.index+84916)
df_preds

In [ ]:
df_preds.to_csv("eval_clip3_5-epoch.csv")